In [10]:
import time
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
import requests
import math
import pandas as pd
from IPython.display import display, HTML
from IPython.display import display_html

class ConsensusFromWeb(object):
    def __init__(self, day):
        self._checkConsensusStatus = 0
        self._isTreasuryClicked = 1
        self._isEbitdaClicked = 2

        self._chrome_path = "C:/Users/jihod/Quant/Kiwoom/Data/chromedriver.exe"
        self._driver = webdriver.Chrome(self._chrome_path)
        self._delay = 10
        self._day = day


    def parse_data(self,stock,menuNum, tabNum,optionNum,isNeedPreProcess=False):
        self._preparationByTab(stock, menuNum,tabNum,optionNum,isNeedPreProcess)
        return self.get_data(stock, menuNum)

    def get_data(self, stock, menuNum):
        try:
            html_source = self._driver.page_source     
            if menuNum==1:
                text_split_first = html_source.split('익계산서,재무상태표,현금흐름표에')[2]
            if menuNum==2:
                text_split_first = html_source.split('데이터를 보여줍니다.')[2]
            

            text_split_second = text_split_first.split("table")[0]

            finalstring = "<table"+text_split_second+"table>"
            finalstring = finalstring.replace("display: none", "display:block")
            crypto_data = pd.read_html(finalstring)
            crypto_data = crypto_data[0]
            crypto_data = crypto_data.rename(
                columns={'항목': '재무'})
            df = crypto_data.set_index('재무')
            colNameList = df.columns.tolist()
            if "2018/03" in colNameList[0]:
                df = df.rename(
                    columns={colNameList[0]: "2018-03-31", colNameList[1]: "2018-06-30",colNameList[2]: "2018-09-30", 
                             colNameList[3]: "2018-12-31", colNameList[4]:'2019-03-31',colNameList[5]:'2019-06-30'})
            elif "2018/06" in colNameList[0]:
                df = df.rename(
                    columns={colNameList[0]: "2018-06-30",colNameList[1]: "2018-09-30", colNameList[2]: "2018-12-31"
                             , colNameList[3]:'2019-03-31',colNameList[4]:'2019-06-30'})
            elif "2018/09" in colNameList[0]:
                df = df.rename(
                    columns={colNameList[0]: "2018-09-30", colNameList[1]: "2018-12-31"
                             , colNameList[2]:'2019-03-31',colNameList[3]:'2019-06-30', colNameList[4]:'2019-09-30'})        
            elif "2018/12" in colNameList[0]:
                df = df.rename(
                    columns={colNameList[0]: "2018-12-31"
                             , colNameList[1]:'2019-03-31',colNameList[2]:'2019-06-30',colNameList[3]:'2019-09-30',\
                             colNameList[4]:'2019-12-31'}) 
            elif "2019/03" in colNameList[0]:
                df = df.rename(
                    columns={colNameList[0]: "2019-03-31"
                             , colNameList[1]:'2019-06-30',colNameList[2]:'2019-09-30',colNameList[3]:'2019-12-31'\
                                 ,colNameList[4]:'2020-03-31'})                  
            self._dropUnneededCol(df)
                
            return df
        except TimeoutException:
            print ("로딩이 제대로 되어지지 않았습니다.")
        return -1

        
    def _preparationByTab(self, stock, menuNum,tabNum, optionNum, isNeedPreProcess):
        ##selenium을 쓰기 위한 초기설정.
        if(menuNum !=1 and menuNum !=2):
            raise Exception("menu 값이 잘못되었습니다 (1~2)")
        if(tabNum !=1 and tabNum !=2 and tabNum !=3):
            raise Exception("tab 값이 잘못되었습니다 (1~3)")
        if(optionNum !=1 and optionNum !=2 and optionNum !=3 and optionNum !=4 and optionNum !=5):
            raise Exception("option 값이 잘못되었습니다 (1~5)")
        if menuNum==1: 
            newUrl = 'https://comp.wisereport.co.kr/company/c1030001.aspx?cmp_cd='+stock[1:]+'&cn='      
            self._driver.get(newUrl)
            if isNeedPreProcess:
                if tabNum ==1:
                    self._driver.find_element_by_xpath('//*[@id="rpt_tab1"]').click()
                if tabNum ==2:
                    self._driver.find_element_by_xpath('//*[@id="rpt_tab2"]').click()
                if tabNum ==3:
                    self._driver.find_element_by_xpath('//*[@id="rpt_tab3"]').click()
        if menuNum==2: 
            newUrl = 'https://comp.wisereport.co.kr/company/c1040001.aspx?cmp_cd='+stock[1:]+'&cn='      
            self._driver.get(newUrl)
            if isNeedPreProcess:
                if tabNum ==1:
                    self._driver.find_element_by_xpath('//*[@id="val_tab1"]').click()
                if tabNum ==2:
                    self._driver.find_element_by_xpath('//*[@id="val_tab2"]').click()
                if tabNum ==3:
                    self._driver.find_element_by_xpath('//*[@id="val_tab3"]').click()
        if isNeedPreProcess:
            self._driver.find_element_by_xpath('//*[@id="finGubun"]/option[' + str(optionNum)+']').click()   
            self._driver.find_element_by_xpath('//*[@id="wrapper"]/table[2]/tbody/tr/td[2]/dl/dt[3]/label').click()
            self._driver.find_element_by_xpath('//*[@id="hfinGubun"]').click()
        myElem = WebDriverWait(self._driver, self._delay).until(EC.presence_of_element_located((By.CLASS_NAME, 'cut')))

       
            
    def _tryGettingData(self, df, 재무지표, 단위=100000000):
        ##데이터 가져오기(없으면 0 처리)
        값 = 0
        try:
            값 = df.at[재무지표]
            if isinstance(값, str):
                if(값=="완전잠식"):
                    return "완전잠식"
            값 = float(값) *단위
            
        except:
            return 0

        
        if math.isnan(값):
            값 = 0
        return 값
    
    def _dropUnneededCol(self, df):
        ##필요없는 컬럼 제거
        colList = df.columns.tolist()
        if 6 in colList:
            df = df.drop(6, axis=1)
        if 7 in colList:
            df = df.drop(7, axis=1)
        if 8 in colList:
            df = df.drop(8, axis=1)

In [14]:
import sqlite3
import requests
class ConsensusToExcelWriter(object):
    def __init__(self, load_wb,load_wb2, write_wb, day):
        ##error list에 대해서 다시 파싱해주는 미구현 기능
        self._errorList = dict()
        self._load_wb = load_wb
        self._load_wb2 = load_wb2
        self._write_wb = write_wb
        self._dest_filename = "C:/Users/jihod/Quant/Kiwoom/Data/dataoverride7.xlsx"
        self._load_ws = self._load_wb['Sheet3']
        self._write_ws = self._write_wb['Sheet3']
        self._load_ws2 = self._load_wb2['Sheet1']

        self._sheet = "Sheet3"
        self._day = day      
        #크롤링 시작점. 재무지표에 따라 바뀜
        self._codeCellItr = 2

        #현재 크롤링중인 지표이름
        self._consensus = ""
        
        #한 재무지표가 끝나면 selenium 세팅 변경
        self._needToChange = False
        
        ##최대 데이타 요청 횟수
        self._defaultTryCount = 5
        
        ##crawler
        self._crawler = ConsensusFromWeb(day)
    def _setConsensus(self, 재무지표):
        self._consensus = 재무지표
        self._needToChange = True
        
        if 재무지표 == "총자산":
            self._codeCellItr = 2
        elif 재무지표 == "총자본":
            self._codeCellItr = 3
        elif 재무지표 == "매출총이익":
            self._codeCellItr = 4
        elif 재무지표 == "지배주주순이익":
            self._codeCellItr = 5
        elif 재무지표 == "매출액":
            self._codeCellItr = 6
        elif 재무지표 == "현금흐름":
            self._codeCellItr = 7
        elif 재무지표 == "부채비율":
            self._codeCellItr = 8
            
        
        
    def _checkSheetChange(self):
        if(self._load_ws.cell(9,self._codeCellItr).value==None):
            self._setConsensus(self._consensus)     
            if(self._sheet=="Sheet4"):
                self._load_ws = self._load_wb['Sheet3']
                self._write_ws = self._write_wb['Sheet3']
                self._load_ws2 = self._load_wb2['Sheet1']

                print(self._consensus + '완료')
                self._sheet ="Sheet3"
                return True
            self._load_ws = self._load_wb['Sheet4']
            self._write_ws = self._write_wb['Sheet4']
            self._load_ws2 = self._load_wb2['Sheet2']

            self._sheet ="Sheet4"
            return False
    
    def write(self, 재무지표, isDoubleCheck = False):
        self._setConsensus(재무지표)
        self._sheet = "Sheet3"
        oldCode = "A005930"
        newCode = ""
        codeCount = 0
        while(True):
            oldCode = newCode

            newCode = self._load_ws.cell(9,self._codeCellItr).value
            
            if isDoubleCheck:
                if newCode not in list(self._errorList.keys()):
                    self._codeCellItr +=7
                    if self._checkSheetChange():
                        break
                else:
                    if 재무지표 not in self._errorList[newCode] :
                        self._codeCellItr +=7
                        if self._checkSheetChange():
                            break
            print(newCode," : ",codeCount," sheet :", self._sheet ," cellWriteLoc row 92 and col: ", self._codeCellItr)
            tryCount = self._defaultTryCount
            if(oldCode != newCode):
                ##접속 문제인 경우 10번까지 시도, 안되면 에러가 있는 종목(상장폐지 등)
                while(True):
                    if(tryCount<0):
                        tryCount = self._defaultTryCount
                        writeValue = None
                        self._needToChange = True
                        break
                    try:
                        writeValue = self._findWriteCSS(재무지표, newCode)
                        self._needToChange = False
                        break
                    except:
                        writeValue = 0
                        print("warning :: Try again-- ", newCode)
                        tryCount-=1
                codeCount+=1

            cellForWrite = self._write_ws.cell(row = 94, column = self._codeCellItr)
            
            if writeValue!=None:
                cellForWrite.value = writeValue

            self._codeCellItr+=7
            self._write_wb.save(filename = self._dest_filename)
            if self._checkSheetChange():
                break
            
    def _findWholeStock(self):
        con = sqlite3.connect("C:/Users/jihod/Quant/Kiwoom/Data/상장종목검색.db")

        cursor = con.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        list_index = cursor.fetchall()
        newList  = []
        for item in list_index:
            newList.append(item[0])
        return newList
       
    def _findStockInFile(self):
        self._sheet = "Sheet3"
        oldCode = "A005930"
        newCode = ""
        codeCount = 0
        list = []
        self._codeCellItr = 2
        while(True):
            oldCode = newCode
            newCode = self._load_ws.cell(9,self._codeCellItr).value
            if(oldCode != newCode):
                list.append(newCode)

            self._codeCellItr+=1
            if self._checkSheetChange():
                break
        return list
    
    # 신규 상장 종목 검색
    def _findNewlyAddedStock(self):  
        url = 'https://finance.naver.com/sise/sise_new_stock.nhn?sosok=0'
        response = requests.get(url)
        text = response.text
        text_split_first = text.split('/item/main.nhn?code=')
        newlyAddedStock = []
        for i in range(99):
            code = "A" + text_split_first[i+1][0:6]
            newlyAddedStock.append(code)
        url = 'https://finance.naver.com/sise/sise_new_stock.nhn?sosok=1'
        response = requests.get(url)
        text = response.text
        text_split_first = text.split('/item/main.nhn?code=')
        for i in range(99):
            code = "A" + text_split_first[i+1][0:6]
            newlyAddedStock.append(code)

        return newlyAddedStock
                
        
    def _findWriteCSS(self, 재무지표, newCode):
        if (재무지표 == '총자산') or (재무지표 =="총자산(원)"):
            df = self._crawler.parse_data(newCode, 1,2,1, self._needToChange)[self._day]
            return self._crawler._tryGettingData(df, '자산총계') +self._crawler._tryGettingData(df, '자산')

        elif 재무지표 == '총자본(원)' or 재무지표 =='총자본':
            df = self._crawler.parse_data(newCode, 1,2,1, self._needToChange)[self._day]
            return self._crawler._tryGettingData(df, '*순자산1')+self._crawler._tryGettingData(df, '자본')

        elif 재무지표 == '매출총이익' or 재무지표 == '매출총이익(원)':
            df = self._crawler.parse_data(newCode, 1,1,1, self._needToChange)[self._day] 
            if "은행" in newCode or "금융" in newCode or "지주" in newCode:
                return self._crawler._tryGettingData(df, '펼치기  순이자이익')
            if "화재" in newCode or "보험" in newCode or "생명" in newCode:
                return self._crawler._tryGettingData(df, '보험료수익') - \
                        self._crawler._tryGettingData(df, '보험계약부채(책임준비금)전입액')
            if "카드" in newCode or "증권" in newCode or "캐피탈" in newCode:
                return self._crawler._tryGettingData(df, '순영업이익')        
            if "신탁" in newCode :
                return self._crawler._tryGettingData(df, '펼치기  영업수익') - \
                        self._crawler._tryGettingData(df, '펼치기  영업비용') + \
                        self._crawler._tryGettingData(df, '판매비와관리비')
            return self._crawler._tryGettingData(df, '매출총이익')

        elif 재무지표 == '지배주주순이익' or 재무지표 == '지배주주순이익(원)': 
            df = self._crawler.parse_data(newCode, 1,1,1, self._needToChange)[self._day] 
            return self._crawler._tryGettingData(df, '(지배주주지분)당기순이익')

        elif 재무지표 == '매출액' or 재무지표 == '매출액(원)' or 재무지표 =='매출액(수익)':
            df = self._crawler.parse_data(newCode, 1,1,1, self._needToChange)[self._day] 
            if "은행" in newCode or "금융" in newCode or "지주" in newCode:
                return self._crawler._tryGettingData(df, '이자수익')
            if "화재" in newCode or "보험" in newCode or "생명" in newCode:
                return self._crawler._tryGettingData(df, '보험료수익')
            if "카드" in newCode or "증권" in newCode or "캐피탈" in newCode:
                return self._crawler._tryGettingData(df, '순영업이익')        
            if "신탁" in newCode :
                return self._crawler._tryGettingData(df, '영업수익')         
            return self._crawler._tryGettingData(df, '펼치기  매출액(수익)')
        
        elif 재무지표 == '현금흐름' or 재무지표 == '현금흐름(원)': 
            retValue = self._load_ws2.cell(row = 94, column = self._codeCellItr-2).value
            return retValue
        elif 재무지표 == '부채비율' or 재무지표 == '부채비율(%)': 
            print("부채 : ", self._load_ws2.cell(row = 94, column = self._codeCellItr-1).value)
            print("자산총계 : ", self._write_ws.cell(row = 94, column = self._codeCellItr-6).value)

            retValue = float(self._load_ws2.cell(row = 94, column = self._codeCellItr-1).value)/\
                        float((self._write_ws.cell(row = 94, column = self._codeCellItr-6).value\
                               -self._load_ws2.cell(row = 94, column = self._codeCellItr-1).value))
            
            print(retValue)
            return retValue*100
        
    def writeNewlyAddedStock(self):
        self._sheet = "Sheet3"
        self._load_ws = self._load_wb["Sheet3"]
        newStockList = self._findNewlyAddedStock()
        fileStockList = self._findStockInFile()
        wholeStockList = self._findWholeStock()
        self._codeCellItr = 2
        unaddedNewStockList = list(set(newStockList)-set(fileStockList))
        
        newStockListFinal = list(set(unaddedNewStockList).intersection(wholeStockList))
        newStockListFinal = sort(newStockListFinal)
        codeCount = 0
        while(True):
            findBlank = self._load_ws.cell(row = 9, column = self._codeCellItr)
            if findBlank.value==None:
                break
            self._codeCellItr+=1
        for item in newStockListFinal:
            print("새로 상장된 종목 추가 : ", item)
            writeNewCode =  self._load_ws.cell(row = 9, column = self._codeCellItr)
            writeNewCode.value = item
            writeNewCode =  self._load_ws.cell(row = 13, column = self._codeCellItr)
            writeNewCode.value = "총자산(원)"

            writeNewCode =  self._load_ws.cell(row = 9, column = self._codeCellItr+1)
            writeNewCode.value = item
            writeNewCode =  self._load_ws.cell(row = 13, column = self._codeCellItr+1)
            writeNewCode.value = "총자본(원)"
            
            writeNewCode =  self._load_ws.cell(row = 9, column = self._codeCellItr+2)
            writeNewCode.value = item
            writeNewCode =  self._load_ws.cell(row = 13, column = self._codeCellItr+2)
            writeNewCode.value = "매출총이익(원)"

            writeNewCode =  self._load_ws.cell(row = 9, column = self._codeCellItr+3)
            writeNewCode.value = item
            writeNewCode =  self._load_ws.cell(row = 13, column = self._codeCellItr+3)
            writeNewCode.value = "지배주주순이익(원)"
            
            writeNewCode =  self._load_ws.cell(row = 9, column = self._codeCellItr+4)
            writeNewCode.value = item
            writeNewCode =  self._load_ws.cell(row = 13, column = self._codeCellItr+4)
            writeNewCode.value = "매출액(원)"
            
            writeNewCode =  self._load_ws.cell(row = 9, column = self._codeCellItr+5)
            writeNewCode.value = item
            writeNewCode =  self._load_ws.cell(row = 13, column = self._codeCellItr+5)
            writeNewCode.value = "현금흐름(원)"
            
            writeNewCode =  self._load_ws.cell(row = 9, column = self._codeCellItr+6)
            writeNewCode.value = item
            writeNewCode =  self._load_ws.cell(row = 13, column = self._codeCellItr+6)
            writeNewCode.value = "부채비율(%)"
            self._codeCellItr+=7

        self._load_wb.save(filename = "C:/Users/jihod/Quant/Kiwoom/Data/재무데이터(분기)(퀄리티).xlsx")
        self._codeCellItr = 2       
        
    def aversionCheck(self, 재무지표, isDoubleCheck=False, errorBound = 0.15):
        self._setConsensus(재무지표)
        oldCode = "A005930"
        self._sheet="Sheet3"
        newCode = ""
        codeCount = 0
        correctDataCount = 0
        while(True):
            oldCode = newCode

            newCode = self._load_ws.cell(9,self._codeCellItr).value

            tryCount = self._defaultTryCount
#             if(oldCode != newCode):

            cellWritten = self._write_ws.cell(row = 92, column = self._codeCellItr)
            cellOrigin = self._load_ws.cell(row = 91, column = self._codeCellItr)
            checkCond = cellWritten.value != None and \
                        cellOrigin.value != None and \
                        cellOrigin.value !='' and \
                        cellWritten.value !='' and \
                        cellOrigin.value !='완전잠식' and \
                        cellWritten.value !='완전잠식'

            if checkCond:
                cellWritten.value = int(cellWritten.value)
                cellOrigin.value = int(cellOrigin.value)
           
                print("cellOrigin.value : " , str(cellOrigin.value))
                print("cellWritten.value : " , str(cellWritten.value))

                if abs(cellWritten.value) != 0:
                    codeCount+=1
                    if cellOrigin.value  == 0:
                        if newCode not in list(self._errorList.keys()):
                            self._errorList[newCode] = [재무지표]
                        else:
                            if 재무지표 not in self._errorList[newCode]:
                                self._errorList[newCode].append(재무지표)
                    elif abs(cellWritten.value - cellOrigin.value)/abs(cellWritten.value) > errorBound:
                        if newCode not in list(self._errorList.keys()):
                            self._errorList[newCode] = [재무지표]
                        else:
                            if 재무지표 not in self._errorList[newCode]:
                                self._errorList[newCode].append(재무지표)
                    elif abs(cellWritten.value - cellOrigin.value)/abs(cellWritten.value) <= errorBound:
                        correctDataCount+=1
                        if isDoubleCheck:
                            if newCode in list(self._errorList.keys()):
                                if 재무지표 in self._errorList[newCode]:
                                    self._errorList[newCode].remove(재무지표)
                                    if len(self._errorList[newCode])==0:
                                        del self._errorList[newCode]
            self._codeCellItr+=7

            if self._checkSheetChange():
                break
        print(재무지표+" 데이타 정합율 : ", str(correctDataCount/ codeCount))

In [15]:
from openpyxl import load_workbook

start = time.time()
_override_path = "C:/Users/jihod/Quant/Kiwoom/Data/dataoverride7.xlsx"
#data_only=Ture로 해줘야 수식이 아닌 값으로 받아온다.
load_wb = load_workbook("C:/Users/jihod/Quant/Kiwoom/Data/재무데이터(분기)(퀄리티).xlsx", data_only=True)
load_wb2 = load_workbook("C:/Users/jihod/Quant/Kiwoom/Data/재무데이터(분기).xlsx", data_only=True)

write_wb = load_workbook(_override_path, data_only=True)

res = len(write_wb.sheetnames)
if res==1:
    write_wb.create_sheet('Sheet4')
    write_wb.save(_override_path)
    write_wb.create_sheet('Sheet2')
    write_wb.save(_override_path)
    write_wb.create_sheet('Sheet3')
    write_wb.save(_override_path)

In [16]:
writer = ConsensusToExcelWriter(load_wb, load_wb2, write_wb, "2019-12-31")

# writer.writeNewlyAddedStock()

# writer.write("총자산")
# # writer.aversionCheck("총자산")


# writer.write("총자본")
# # writer.aversionCheck("총자본")

# writer.write("매출총이익")
# # writer.aversionCheck("매출총이익")
# writer.write("지배주주순이익")
# writer.aversionCheck("지배주주순이익")

# writer.write("매출액")
# # writer.aversionCheck("매출액")

# writer.write("현금흐름")
# # writer.aversionCheck("현금흐름")

writer.write("부채비율")
# writer.aversionCheck("부채비율")


A005930  :  0  sheet : Sheet3  cellWriteLoc row 92 and col:  6
A000660  :  1  sheet : Sheet3  cellWriteLoc row 92 and col:  13
A005380  :  2  sheet : Sheet3  cellWriteLoc row 92 and col:  20
A068270  :  3  sheet : Sheet3  cellWriteLoc row 92 and col:  27
A051910  :  4  sheet : Sheet3  cellWriteLoc row 92 and col:  34
A055550  :  5  sheet : Sheet3  cellWriteLoc row 92 and col:  41
A012330  :  6  sheet : Sheet3  cellWriteLoc row 92 and col:  48
A017670  :  7  sheet : Sheet3  cellWriteLoc row 92 and col:  55
A005490  :  8  sheet : Sheet3  cellWriteLoc row 92 and col:  62
A051900  :  9  sheet : Sheet3  cellWriteLoc row 92 and col:  69
A207940  :  10  sheet : Sheet3  cellWriteLoc row 92 and col:  76
A105560  :  11  sheet : Sheet3  cellWriteLoc row 92 and col:  83
A035420  :  12  sheet : Sheet3  cellWriteLoc row 92 and col:  90
A028260  :  13  sheet : Sheet3  cellWriteLoc row 92 and col:  97
A034730  :  14  sheet : Sheet3  cellWriteLoc row 92 and col:  104
A000270  :  15  sheet : Sheet3  cel

A004370  :  121  sheet : Sheet3  cellWriteLoc row 92 and col:  853
A010120  :  122  sheet : Sheet3  cellWriteLoc row 92 and col:  860
A181710  :  123  sheet : Sheet3  cellWriteLoc row 92 and col:  867
A003240  :  124  sheet : Sheet3  cellWriteLoc row 92 and col:  874
A009420  :  125  sheet : Sheet3  cellWriteLoc row 92 and col:  881
A161890  :  126  sheet : Sheet3  cellWriteLoc row 92 and col:  888
A051600  :  127  sheet : Sheet3  cellWriteLoc row 92 and col:  895
A139130  :  128  sheet : Sheet3  cellWriteLoc row 92 and col:  902
A006280  :  129  sheet : Sheet3  cellWriteLoc row 92 and col:  909
A204320  :  130  sheet : Sheet3  cellWriteLoc row 92 and col:  916
A214320  :  131  sheet : Sheet3  cellWriteLoc row 92 and col:  923
A042670  :  132  sheet : Sheet3  cellWriteLoc row 92 and col:  930
A005300  :  133  sheet : Sheet3  cellWriteLoc row 92 and col:  937
A020560  :  134  sheet : Sheet3  cellWriteLoc row 92 and col:  944
A000670  :  135  sheet : Sheet3  cellWriteLoc row 92 and col: 

A003230  :  242  sheet : Sheet3  cellWriteLoc row 92 and col:  1700
A005690  :  243  sheet : Sheet3  cellWriteLoc row 92 and col:  1707
A001720  :  244  sheet : Sheet3  cellWriteLoc row 92 and col:  1714
A271980  :  245  sheet : Sheet3  cellWriteLoc row 92 and col:  1721
A003850  :  246  sheet : Sheet3  cellWriteLoc row 92 and col:  1728
A004430  :  247  sheet : Sheet3  cellWriteLoc row 92 and col:  1735
A002990  :  248  sheet : Sheet3  cellWriteLoc row 92 and col:  1742
A019680  :  249  sheet : Sheet3  cellWriteLoc row 92 and col:  1749
A053210  :  250  sheet : Sheet3  cellWriteLoc row 92 and col:  1756
A010050  :  251  sheet : Sheet3  cellWriteLoc row 92 and col:  1763
A108670  :  252  sheet : Sheet3  cellWriteLoc row 92 and col:  1770
A025860  :  253  sheet : Sheet3  cellWriteLoc row 92 and col:  1777
A298050  :  254  sheet : Sheet3  cellWriteLoc row 92 and col:  1784
A007570  :  255  sheet : Sheet3  cellWriteLoc row 92 and col:  1791
A000370  :  256  sheet : Sheet3  cellWriteLoc ro

A000020  :  360  sheet : Sheet3  cellWriteLoc row 92 and col:  2526
A026890  :  361  sheet : Sheet3  cellWriteLoc row 92 and col:  2533
A161000  :  362  sheet : Sheet3  cellWriteLoc row 92 and col:  2540
A095570  :  363  sheet : Sheet3  cellWriteLoc row 92 and col:  2547
A001940  :  364  sheet : Sheet3  cellWriteLoc row 92 and col:  2554
A001200  :  365  sheet : Sheet3  cellWriteLoc row 92 and col:  2561
A004710  :  366  sheet : Sheet3  cellWriteLoc row 92 and col:  2568
A267850  :  367  sheet : Sheet3  cellWriteLoc row 92 and col:  2575
A000680  :  368  sheet : Sheet3  cellWriteLoc row 92 and col:  2582
A079980  :  369  sheet : Sheet3  cellWriteLoc row 92 and col:  2589
A036580  :  370  sheet : Sheet3  cellWriteLoc row 92 and col:  2596
A004700  :  371  sheet : Sheet3  cellWriteLoc row 92 and col:  2603
A015360  :  372  sheet : Sheet3  cellWriteLoc row 92 and col:  2610
A272550  :  373  sheet : Sheet3  cellWriteLoc row 92 and col:  2617
A092230  :  374  sheet : Sheet3  cellWriteLoc ro

A013570  :  478  sheet : Sheet3  cellWriteLoc row 92 and col:  3352
A001460  :  479  sheet : Sheet3  cellWriteLoc row 92 and col:  3359
A004200  :  480  sheet : Sheet3  cellWriteLoc row 92 and col:  3366
A002760  :  481  sheet : Sheet3  cellWriteLoc row 92 and col:  3373
A017900  :  482  sheet : Sheet3  cellWriteLoc row 92 and col:  3380
A005950  :  483  sheet : Sheet3  cellWriteLoc row 92 and col:  3387
A008970  :  484  sheet : Sheet3  cellWriteLoc row 92 and col:  3394
A000390  :  485  sheet : Sheet3  cellWriteLoc row 92 and col:  3401
A000230  :  486  sheet : Sheet3  cellWriteLoc row 92 and col:  3408
A003460  :  487  sheet : Sheet3  cellWriteLoc row 92 and col:  3415
A024090  :  488  sheet : Sheet3  cellWriteLoc row 92 and col:  3422
A000890  :  489  sheet : Sheet3  cellWriteLoc row 92 and col:  3429
A267290  :  490  sheet : Sheet3  cellWriteLoc row 92 and col:  3436
A014790  :  491  sheet : Sheet3  cellWriteLoc row 92 and col:  3443
A000490  :  492  sheet : Sheet3  cellWriteLoc ro

...

A068630  :  2989  sheet : Sheet4  cellWriteLoc row 92 and col:  12902
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
A068770  :  2990  sheet : Sheet4  cellWriteLoc row 92 and col:  12909
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
A071360  :  2991  sheet : Sheet4  cellWriteLoc row 92 and col:  12916
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
A071530  :  2992  sheet : Sheet4  cellWriteLoc row 92 and col:  12923
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
A071660  :  2993  sheet : Sheet4  cellWriteLoc row 92 and col:  12930
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채

A079870  :  3010  sheet : Sheet4  cellWriteLoc row 92 and col:  13049
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
A080140  :  3011  sheet : Sheet4  cellWriteLoc row 92 and col:  13056
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
A080570  :  3012  sheet : Sheet4  cellWriteLoc row 92 and col:  13063
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
A081090  :  3013  sheet : Sheet4  cellWriteLoc row 92 and col:  13070
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
A081220  :  3014  sheet : Sheet4  cellWriteLoc row 92 and col:  13077
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채

A089480  :  3031  sheet : Sheet4  cellWriteLoc row 92 and col:  13196
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
A090090  :  3032  sheet : Sheet4  cellWriteLoc row 92 and col:  13203
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
A090120  :  3033  sheet : Sheet4  cellWriteLoc row 92 and col:  13210
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
A090730  :  3034  sheet : Sheet4  cellWriteLoc row 92 and col:  13217
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
A091690  :  3035  sheet : Sheet4  cellWriteLoc row 92 and col:  13224
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채

A124050  :  3052  sheet : Sheet4  cellWriteLoc row 92 and col:  13343
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
A126680  :  3053  sheet : Sheet4  cellWriteLoc row 92 and col:  13350
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
A128910  :  3054  sheet : Sheet4  cellWriteLoc row 92 and col:  13357
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
A130960  :  3055  sheet : Sheet4  cellWriteLoc row 92 and col:  13364
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
A155960  :  3056  sheet : Sheet4  cellWriteLoc row 92 and col:  13371
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채

A221200  :  3073  sheet : Sheet4  cellWriteLoc row 92 and col:  13490
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
A221950  :  3074  sheet : Sheet4  cellWriteLoc row 92 and col:  13497
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
A222390  :  3075  sheet : Sheet4  cellWriteLoc row 92 and col:  13504
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
A223040  :  3076  sheet : Sheet4  cellWriteLoc row 92 and col:  13511
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
A225440  :  3077  sheet : Sheet4  cellWriteLoc row 92 and col:  13518
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채 :  None
자산총계 :  None
부채